# Forecasting Tourism Demand in Singapore

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Collection and Preprocessing

### Importing Dataset

In [9]:
inter_arr = 'data/raw/international_visitor_arrivals.csv'
inter_arr_stay_days = 'data/raw/international_visitor_arrivals_stay_days.csv'
inter_arr_age = 'data/raw/international_visitor_arrivals_age.csv'
hotel_stats = 'data/raw/hotel_statistics.csv'
weather = 'data/raw/weather.csv'
purpose_visit = 'data/raw/purpose_of_visit.xlsx'
spent = 'data/raw/money_spent.xlsx'
holiday_14_22 = 'data/raw/holidays_2014_2022.csv'
holiday_23 = 'data/raw/holidays_2023.csv'
holiday_24 = 'data/raw/holidays_2024.csv'

inter_arrival_df = pd.read_csv(inter_arr)
inter_arrival_stay_days_df = pd.read_csv(inter_arr_stay_days)
inter_arrival_age_df = pd.read_csv(inter_arr_age)
hotel_stats_df = pd.read_csv(hotel_stats)
weather_df = pd.read_csv(weather)
purpose_visit_df = pd.read_excel(purpose_visit)
spent_df = pd.read_excel(spent)
holiday_14_22_df = pd.read_csv(holiday_14_22)
holiday_23_df = pd.read_csv(holiday_23)
holiday_24_df = pd.read_csv(holiday_24)

### Converting long to wide format and Cleaning

In [3]:
# International Visitor Arrival
inter_arrival_df = inter_arrival_df.melt(id_vars='DataSeries', var_name='date', value_name='visitor_arrivals')
inter_arrival_df['date'] = pd.to_datetime(inter_arrival_df['date'], format="%Y%b")
inter_arrival_df.set_index('date', inplace=True)
inter_arrival_df.rename(columns={
    'DataSeries': 'region'
} ,inplace=True)
inter_arrival_df = inter_arrival_df.loc['2015-01-01':]
inter_arrival_df['region'] = inter_arrival_df['region'].str.replace('Total International Visitor Arrivals By Inbound Tourism Markets', 'Total')

In [4]:
# International Visitor Arrival Stay Days
inter_arrival_stay_days_df = inter_arrival_stay_days_df.melt(id_vars='DataSeries', var_name='date', value_name='visitor_arrivals')
inter_arrival_stay_days_df['date'] = pd.to_datetime(inter_arrival_stay_days_df['date'], format="%Y%b")
inter_arrival_stay_days_df.set_index('date', inplace=True)
inter_arrival_stay_days_df.rename(columns={
    'DataSeries': 'stay_days'
} ,inplace=True)
inter_arrival_stay_days_df = inter_arrival_stay_days_df.loc['2015-01-01':]
inter_arrival_stay_days_df['stay_days'] = inter_arrival_stay_days_df['stay_days'].str.replace('Total International Visitor Arrivals', 'Total')

In [5]:
# International Visitor Arrival Age
inter_arrival_age_df = inter_arrival_age_df.melt(id_vars='DataSeries', var_name='date', value_name='visitor_arrivals')
inter_arrival_age_df['date'] = pd.to_datetime(inter_arrival_age_df['date'], format="%Y%b")
inter_arrival_age_df.set_index('date', inplace=True)
inter_arrival_age_df.rename(columns={
    'DataSeries': 'age'
} ,inplace=True)
inter_arrival_age_df = inter_arrival_age_df.loc['2015-01-01':]

In [6]:
# International Visitor Arrival Gender
inter_arrival_gender_df = inter_arrival_age_df[inter_arrival_age_df['age'].isin(['Males','Females'])]
inter_arrival_age_df = inter_arrival_age_df[~inter_arrival_age_df['age'].isin(['Males','Females'])]

In [7]:
# Hotel Statistics
hotel_stats_df = hotel_stats_df.melt(id_vars='DataSeries', var_name='date', value_name='statistics')
hotel_stats_df['date'] = pd.to_datetime(hotel_stats_df['date'], format="%Y%b")
hotel_stats_df.set_index('date', inplace=True)
hotel_stats_df.rename(columns={
    'DataSeries': 'hotels_data'
} ,inplace=True)
hotel_stats_df = hotel_stats_df.loc['2015-01-01':]

In [8]:
# Weather
weather_df = weather_df.melt(id_vars='DataSeries', var_name='date', value_name='weather_data')
weather_df['date'] = pd.to_datetime(weather_df['date'], format="%Y%b")
weather_df.set_index('date', inplace=True)
weather_df.rename(columns={
    'DataSeries': 'weather_info'
} ,inplace=True)
weather_df = weather_df.loc['2015-01-01':]

In [9]:
# Purpose of Visit
purpose_visit_df = purpose_visit_df.melt(id_vars='DataSeries', var_name='date', value_name='count')
purpose_visit_df['date'] = pd.to_datetime(purpose_visit_df['date'], format="%Y").dt.year
purpose_visit_df.set_index('date', inplace=True)
purpose_visit_df.rename(columns={
    'DataSeries': 'purpose'
} ,inplace=True)
purpose_visit_df = purpose_visit_df.loc['2010':]
purpose_visit_df.index = purpose_visit_df.index.sort_values(ascending=False)

In [10]:
# Purpose of Visit
spent_df = spent_df.melt(id_vars='DataSeries', var_name='date', value_name='dollar(millions)')
spent_df['date'] = pd.to_datetime(spent_df['date'], format="%Y").dt.year
spent_df.set_index('date', inplace=True)
spent_df.rename(columns={
    'DataSeries': 'travel'
} ,inplace=True)
spent_df = spent_df.loc['2010':]
spent_df.index = spent_df.index.sort_values(ascending=False)
spent_df = spent_df.drop(columns=['travel'])

In [15]:
# Holidays
holiday_14_22_df['date'] = pd.to_datetime(holiday_14_22_df['date'], format='%Y-%m-%d')
holiday_23_df['date'] = pd.to_datetime(holiday_23_df['date'], format='%Y-%m-%d')
holiday_24_df['date'] = pd.to_datetime(holiday_24_df['date'], format='%Y-%m-%d')
holidays = pd.concat([holiday_14_22_df, holiday_23_df, holiday_24_df])
holidays = holidays.reset_index(drop=True)

### Dropping unnecessary columns

In [147]:
unemploy_sex_age_edu_df = unemploy_sex_age_edu_df.drop(['source.label', 'indicator.label', 'obs_status.label', 'note_classif.label', 'note_indicator.label', 'note_source.label'], axis=1)
unemploy_sex_age_marital_df = unemploy_sex_age_marital_df.drop(['source.label', 'indicator.label', 'obs_status.label', 'note_classif.label', 'note_indicator.label', 'note_source.label'], axis=1)
unemploy_sex_age_region_df = unemploy_sex_age_region_df.drop(['source.label', 'indicator.label', 'obs_status.label', 'note_classif.label', 'note_indicator.label', 'note_source.label'], axis=1)

In [155]:
countries = ['India', 'Pakistan']
population_df = population_df[population_df['Country Name'].isin(countries)]
years = [str(year) for year in range(2015,2024)]
population_df = population_df.loc[:,['Country Name']+years]

inflation_df = inflation_df[inflation_df['Country Name'].isin(countries)]
years = [str(year) for year in range(2015,2024)]
inflation_df = inflation_df.loc[:,['Country Name']+years]

gdp_df = gdp_df[gdp_df['Country Name'].isin(countries)]
years = [str(year) for year in range(2015,2024)]
gdp_df = gdp_df.loc[:,['Country Name']+years]

### Renaming Columns

In [156]:
unemploy_sex_age_edu_df.rename(columns={
    'ref_area.label': 'country',
    'sex.label': 'gender',
    'classif1.label': 'age_bracket',
    'classif2.label': 'education',
    'obs_value': 'value'
}, inplace=True)

unemploy_sex_age_marital_df.rename(
    columns={
        'ref_area.label': 'country',
        'sex.label': 'gender',
        'classif1.label': 'age_bracket',
        'classif2.label': 'marital_status',
        'obs_value': 'value'
    }, inplace=True
)

unemploy_sex_age_region_df.rename(
    columns={
        'ref_area.label': 'country',
        'sex.label': 'gender',
        'classif1.label': 'age_bracket',
        'classif2.label': 'region',
        'obs_value': 'value'
    }, inplace=True
)

population_df.rename(columns={'Country Name':'country'}, inplace=True)
inflation_df.rename(columns={'Country Name':'country'}, inplace=True)
gdp_df.rename(columns={'Country Name':'country'}, inplace=True)

### Handling rows and null values

#### Unemployment (Gender, Age, Education) Dataset

In [150]:
unemploy_sex_age_edu_df['gender'] = unemploy_sex_age_edu_df['gender'].str.replace('Sex: ','')
unemploy_sex_age_marital_df['gender'] = unemploy_sex_age_marital_df['gender'].str.replace('Sex: ', '')
unemploy_sex_age_region_df['gender'] = unemploy_sex_age_region_df['gender'].str.replace('Sex: ', '')

unemploy_sex_age_edu_df['age_bracket'] = unemploy_sex_age_edu_df['age_bracket'].str.replace('Age (Aggregate bands): ','')
unemploy_sex_age_marital_df['age_bracket'] = unemploy_sex_age_marital_df['age_bracket'].str.replace('Age (Aggregate bands): ','')
unemploy_sex_age_region_df['age_bracket'] = unemploy_sex_age_region_df['age_bracket'].str.replace('Age (Aggregate bands): ','')

unemploy_sex_age_edu_df['education'] = unemploy_sex_age_edu_df['education'].str.replace('Education (Aggregate levels): ','')

unemploy_sex_age_marital_df['marital_status'] = unemploy_sex_age_marital_df['marital_status'].str.replace('Marital status (Detailed): ',  '')

unemploy_sex_age_region_df['region'] = unemploy_sex_age_region_df['region'].str.replace('Area type: ',  '')

In [151]:
education_to_replace = {
    'Less than basic': 'Below High School',
    'Basic': 'High School', 
    'Intermediate': 'Bachelor\'s',
    'Advanced': 'Master\'s or Higher',
    'Level not stated': 'Not Stated'
}

unemploy_sex_age_edu_df['education'] = unemploy_sex_age_edu_df['education'].replace(education_to_replace)

##### Null Values Handle

In [152]:
unemploy_sex_age_edu_df['value'] = unemploy_sex_age_edu_df.groupby(['age_bracket', 'education'])['value'].transform(lambda x: x.fillna(x.mean()))
unemploy_sex_age_edu_df_value_global_mean = unemploy_sex_age_edu_df['value'].mean()
unemploy_sex_age_edu_df['value'] = unemploy_sex_age_edu_df['value'].fillna(unemploy_sex_age_edu_df_value_global_mean)
unemploy_sex_age_edu_df['value'] = round(unemploy_sex_age_edu_df['value'])

unemploy_sex_age_marital_df['value'] = unemploy_sex_age_marital_df.groupby(['age_bracket', 'marital_status'])['value'].transform(lambda x: x.fillna(x.mean()))
unemploy_sex_age_marital_df_value_global_mean = unemploy_sex_age_marital_df['value'].mean()
unemploy_sex_age_marital_df['value'] = unemploy_sex_age_marital_df['value'].fillna(unemploy_sex_age_marital_df_value_global_mean)
unemploy_sex_age_marital_df['value'] = round(unemploy_sex_age_marital_df['value'])

unemploy_sex_age_region_df['value'] = unemploy_sex_age_marital_df.groupby(['age_bracket', 'marital_status'])['value'].transform(lambda x: x.fillna(x.mean()))
unemploy_sex_age_region_df_value_global_mean = unemploy_sex_age_region_df['value'].mean()
unemploy_sex_age_region_df['value'] = unemploy_sex_age_region_df['value'].fillna(unemploy_sex_age_region_df_value_global_mean)
unemploy_sex_age_region_df['value'] = round(unemploy_sex_age_region_df['value'])

##### Convert Data Types

In [153]:
unemploy_sex_age_edu_df['value'] = unemploy_sex_age_edu_df['value'].astype('int')
unemploy_sex_age_edu_df['gender'] = unemploy_sex_age_edu_df['gender'].astype('category')
unemploy_sex_age_edu_df['age_bracket'] = unemploy_sex_age_edu_df['age_bracket'].astype('category')
unemploy_sex_age_edu_df['education'] = unemploy_sex_age_edu_df['education'].astype('category')
unemploy_sex_age_edu_df['time'] =  pd.PeriodIndex(unemploy_sex_age_edu_df['time'], freq='Q')

unemploy_sex_age_marital_df['value'] = unemploy_sex_age_marital_df['value'].astype('int')
unemploy_sex_age_marital_df['gender'] = unemploy_sex_age_marital_df['gender'].astype('category')
unemploy_sex_age_marital_df['age_bracket'] = unemploy_sex_age_marital_df['age_bracket'].astype('category')
unemploy_sex_age_marital_df['marital_status'] = unemploy_sex_age_marital_df['marital_status'].astype('category')
unemploy_sex_age_marital_df['time'] =  pd.PeriodIndex(unemploy_sex_age_marital_df['time'], freq='Q')

unemploy_sex_age_region_df['value'] = unemploy_sex_age_region_df['value'].astype('int')
unemploy_sex_age_region_df['gender'] = unemploy_sex_age_region_df['gender'].astype('category')
unemploy_sex_age_region_df['age_bracket'] = unemploy_sex_age_region_df['age_bracket'].astype('category')
unemploy_sex_age_region_df['region'] = unemploy_sex_age_region_df['region'].astype('category')
unemploy_sex_age_region_df['time'] =  pd.PeriodIndex(unemploy_sex_age_region_df['time'], freq='Q')